Методы Монте-Карло (ММК) — группа численных методов для изучения случайных процессов. Суть метода заключается в следующем: процесс описывается математической моделью с использованием генератора случайных величин, модель многократно обсчитывается, на основе полученных данных вычисляются вероятностные характеристики рассматриваемого процесса.

In [ ]:
import pandas as pd
import numpy as np

Вероятность — степень (относительная мера, количественная оценка) возможности наступления некоторого события. 

Если бросать кубик множество раз, то тот процент бросков, в которых выпала, например, шестерка - это и есть ее вероятность.

In [ ]:
# вероятность можно оценивать экспериментально
# оценим вероятность выпадения 6-ки на кубике
dice = pd.Series([1,2,3,4,5,6]) # кубик-столбец датафрейма

In [ ]:
# один бросок кубика
# выбор одного случайного сэмпла (значения в столбце)
# replace=True - выборка из одного элемента с возвращением
dice.sample(1, replace=True).values[0]

4

In [ ]:
# броски кубика
rolls = [dice.sample(1, replace=True).values[0] for _ in range(100000)]
rolls[:10]

[5, 5, 6, 6, 2, 5, 2, 6, 3, 2]

In [ ]:
# посчтав, в скольки проценах случаев выпала шестерка в листе rolls, оценим вероятность этого события
pd.Series(rolls).value_counts(normalize=True) # normalize как раз берет долю

4    0.16849
6    0.16736
5    0.16706
1    0.16640
2    0.16543
3    0.16526
dtype: float64

In [ ]:
# аналитически:
1/6

0.16666666666666666

# Задачки

Парадокс дней рождений

Определить вероятность того, что в группе из 23 людей у двоих из них совпадет день рождения (день, месяц)

Интуиция:

Если собирать группы по 23 человека множество раз и каждый раз определять совпадение дней рождений, в какой доле случаев совпадение будет найдено?

In [ ]:
# генератор комнаты с 23 людьми с какими-то днями рождения

bd = pd.Series(range(365)) # возможные дни рождения 0-364
# bd - огромный кубик, который мы бросаем, сажая людей в комнату,
# с возвращением, ибо др могут повторяться

In [ ]:
t = bd.sample(23, replace=True) # сэмплируем 23 человека с возвращением

In [ ]:
t.duplicated() # есть ли совпадения

261    False
82     False
232    False
149    False
181    False
346    False
89     False
362    False
263    False
310    False
131    False
204    False
294    False
82      True
342    False
187    False
97     False
166    False
191    False
355    False
59     False
328    False
164    False
dtype: bool

In [ ]:
t.duplicated().max() # схлопываем до одного значения

True

In [ ]:
# теперь делаем множество комнат
rooms = [bd.sample(23, replace=True).duplicated().max() for _ in range(10000)]

In [ ]:
rooms[:10]

[False, True, False, False, False, True, True, True, True, True]

In [ ]:
# доля комнат, в которых было совпадение
np.mean(rooms)

0.5038

Экзамен - билеты убираются по одному в сторону, после того, как ученики их тянут. Ученик выучил 20 билетов из 30. Когда ему выгоднее идти, 1м, 2м или 20м, чтобы вероятность вытянуть выученный билет была выше?

In [ ]:
tickets = list(range(1, 31)) # 30 билетов

In [ ]:
student = list(range(1, 21)) # студент знает 20

In [ ]:
# перемешаем билеты
from random import shuffle
shuffle(tickets)

In [ ]:
tickets[:5] # какой билет выпал бы студенту, если бы он пошел 1м, 2м и тд

[21, 9, 10, 27, 18]

In [ ]:
# определим, выпал ли студенту тот билет, который он знал
n = 10000 # количество экзаменов
tickets = list(range(1, 31)) # 30 билетов
student = list(range(1, 21)) # студент знает 20

all_results_mean = []

for m in range(20):
  print(m)
  result = [] # сдал или не сдал экзамен
  for _ in range(n):
    shuffle(tickets)
    result.append(tickets[m] in student) # студент знает или нет первый билет
  all_results_mean.append(np.mean(result))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
all_results_mean[:10] # не важно, каким по счету он пойдет

[0.6664,
 0.6667,
 0.6607,
 0.6619,
 0.6682,
 0.6759,
 0.6674,
 0.6616,
 0.6669,
 0.6667]

In [ ]:
# аналитически:
2/3

0.6666666666666666

Произошло дтп с участием такси. Такси в городе синие и зеленые. Зеленым принадлежит 85% всех такси, синим 15%. Свидетель говорит, что такси было синим. Свидетель верно определяет такси в 80% случаев. Какова вероятность, что такси действительно было синим?

In [ ]:
# машина синяя = 1
# генератор, который в 15% случаев дает 1

np.random.binomial(1, 0.15)

1

In [ ]:
def witness(taxi): # свидетель
  if np.random.binomial(1, 0.8): # условие срабатывает в 80% случаев
    return taxi
  return abs(taxi-1)

In [ ]:
witness(0)

0

In [ ]:
# моделируем много дтп
n = 10000
result = []
for _ in range(n):
  taxi = np.random.binomial(1, 0.15) # генерится какое-то такси
  witness_answer = witness(taxi)
  result.append((taxi, witness_answer))

In [ ]:
t = pd.DataFrame(result, columns = ['taxi', 'witness_answer'])

In [ ]:
t

,taxi,witness_answer
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
9995,0,0
9996,0,0
9997,0,0
9998,1,1


In [ ]:
# вероятность, что такси синее
# сгруппируем по ответу свидетеля
t.groupby('witness_answer')['taxi'].mean()

witness_answer
0    0.042341
1    0.408163
Name: taxi, dtype: float64

Русска рулетка. В барабане два патрона подряд. первый человек стреляет и остается жив. второму предлагают покрутить барабан перед выстрелом или сразу стрелять. что выбрать?

In [ ]:
# модель барабана револьвера
chamber = [0,0,0,0,1,1]

In [ ]:
def one_turn(chamber): # поворот барабана на 1 ячейку вперед
  new_chamber = [0,0,0,0,0,0]
  for i in range(len(chamber)):
    new_chamber[(i+1) % len(chamber)] = chamber[i]
  return new_chamber

In [ ]:
one_turn(chamber)

[1, 0, 0, 0, 0, 1]

In [ ]:
# заряжаем
chamber = [1,1,0,0,0,0]

In [ ]:
# первый игрок крутит барабан
# просто случайное количество раз делаем поворот на 1 деление
def spin_chamber(chamber):
  n = np.random.randint(1,7)
  for i in range(n):
    chamber = one_turn(chamber)
  return chamber

In [ ]:
# проверим, равновероятен ли каждый результат
pd.Series([str(spin_chamber(chamber)) for _ in range(10000)]).value_counts()

[0, 0, 1, 1, 0, 0]    1699
[1, 1, 0, 0, 0, 0]    1691
[0, 0, 0, 0, 1, 1]    1683
[0, 0, 0, 1, 1, 0]    1658
[0, 1, 1, 0, 0, 0]    1652
[1, 0, 0, 0, 0, 1]    1617
dtype: int64

In [ ]:
# вращает
chamber = spin_chamber(chamber)

In [ ]:
# первый игрок стеляет
chamber = one_turn(chamber)
# смотрим что выпало в первое гнезд барабана
player1 = chamber[0] # если 1, то застрелен
chamber[0] = 0 # пули в гнезде больше нет
# если первый выжил, второй игрок вращает барабан - нужно это?
chamber = spin_chamber(chamber)
# второй игрок стреляет
chamber = one_turn(chamber)
player2 = chamber[0]

print(player1)
print(player2)


0
1


In [ ]:
# сделаем экспериенты
n = 10000
result = []
for _ in range(n):
  # заряжаем
  chamber = [1,1,0,0,0,0]
  # первый игрок крутит барабан  
  chamber = spin_chamber(chamber)
  # первый игрок стеляет
  chamber = one_turn(chamber)
  # смотрим что выпало в первое гнезд барабана
  player1 = chamber[0] # если 1, то застрелен
  chamber[0] = 0 # пули в гнезде больше нет

  # если первый выжил, второй игрок вращает барабан - нужно это?
  #chamber = spin_chamber(chamber)
  
  # второй игрок стреляет
  chamber = one_turn(chamber)
  player2 = chamber[0]

  result.append((player1, player2))

In [ ]:
t = pd.DataFrame(result, columns = ['player1', 'player2'])

In [ ]:
t

,player1,player2
0,1,1
1,1,0
2,0,1
3,0,1
4,0,0
...,...,...
9995,1,0
9996,0,0
9997,0,0
9998,0,1


In [ ]:
t.groupby('player1')['player2'].agg(['count', 'mean'])

,count,mean
player1,,
0,6664,0.252101
1,3336,0.500600


In [ ]:
# когда игрок 1выжил (строка 0), вероятность второго игрока
# умереть (среднее по выборке - доля единиц):
# 0,33 с вращением барабана
# 0,25 без вращения